# Pandas + dates + times

Mirrored at: https://github.com/reuven/SFPython-2021-08Aug-18

# Agenda

1. Date/time concepts and data structures
2. Timestamp columns (how to read them)
3. Time delta columns (what they are, and how to work with them)
4. Date/time indexes on data frames
5. Time series and resampling


# Time and date concepts

There are basically two of them:

- One moment in time (when the meeting starts, or when you were born) -- timestamp/datetime, representing one specific moment
- A span of time (how long the meeting goes, or how long you have been alive) -- timedelta -- doesn't have a specific start or end, but rather is a length of time

In [1]:
import pandas as pd
from pandas import Series, DataFrame

In [ ]:
df = pd.read_csv('taxi.csv')